In [1]:
#data prefix for output data
yearmonth = "202405"

In [2]:
#import necessary packages
import platform
import os
import sys
import time
#for managing the data
import pandas as pd

#install !pip install webdriver-manager
#webdrivers
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup

import pathlib

import os
from io import StringIO

from tqdm import tqdm

In [3]:
#get current path
curr_path = os.getcwd()
project_tag = curr_path.split("/")[-1]
grouping = curr_path.split("/")[-2]
output_path = "../../../data/{}/{}/{}/".format(grouping,project_tag,yearmonth )

#create data output path if it doesn't exist
pathlib.Path(output_path).mkdir(parents=True, exist_ok=True) 

In [6]:
#load in level 1 data
df = pd.read_csv(output_path + "1_level_table.csv")
print(df.shape)

(1496, 12)


In [7]:
#open web driver
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

In [8]:
#place holder for final dataframe list
df_list = []

for i,r in tqdm( df.iterrows() ):
    url = r['url_entity_partial']
    url_entity = r['url_entity_full']
    
    #load the main landing page
    driver.get(url_entity)
    
    try:
        #find disclaimer button
        disclamer_btn = driver.find_element(by='name', value="AcceptDisclaimerButton")
    
        #click the disclaimer button to move to the data page
        disclamer_btn.click()
    except:
        pass

    #place holder for extracted data
    data_url_list = []
    data_url_full_list = []
    
    #locate each table rows
    data_rows = driver.find_elements(By.CLASS_NAME, value="hovereffect")

    #if no inspection data skip to the next url
    if len(data_rows)==0:
        continue
    
    for row in data_rows:
        data_url_list.append( row.get_attribute("onclick") )
    
    url_prefix = "https://inspectourcommunity.hnhss.ca"
    for u in data_url_list:
        temp_url = url_prefix+u.split("'")[1]
        data_url_full_list.append(temp_url)
    
    tables = pd.read_html(StringIO((driver.page_source)))
    table  = tables[1]
    table['url_inspection'] = data_url_list
    table['url_inspection_full'] = data_url_full_list
    table['url_entity_full'] = url_entity

    df_list.append(table)
 

1496it [08:07,  3.07it/s]


In [9]:
final_df = pd.concat(df_list)
print(final_df.shape)

(3719, 7)


In [10]:
final_df

,Critical Infractions Count,Non Critical Infractions Count,Inspection Type,Inspection Date,url_inspection,url_inspection_full,url_entity_full
0,0,1,Required,30-Jun-2023,location.href = '/Portal/Inspection/Details/9d...,https://inspectourcommunity.hnhss.ca/Portal/In...,https://inspectourcommunity.hnhss.ca/Portal/Fa...
0,1,1,Required,14-Mar-2024,location.href = '/Portal/Inspection/Details/3c...,https://inspectourcommunity.hnhss.ca/Portal/In...,https://inspectourcommunity.hnhss.ca/Portal/Fa...
1,0,0,Required,08-Feb-2023,location.href = '/Portal/Inspection/Details/0e...,https://inspectourcommunity.hnhss.ca/Portal/In...,https://inspectourcommunity.hnhss.ca/Portal/Fa...
2,0,1,Required,08-Sep-2022,location.href = '/Portal/Inspection/Details/e3...,https://inspectourcommunity.hnhss.ca/Portal/In...,https://inspectourcommunity.hnhss.ca/Portal/Fa...
0,1,1,Required,26-Jan-2024,location.href = '/Portal/Inspection/Details/cf...,https://inspectourcommunity.hnhss.ca/Portal/In...,https://inspectourcommunity.hnhss.ca/Portal/Fa...
...,...,...,...,...,...,...,...
14,5,6,Required,22-Nov-2022,location.href = '/Portal/Inspection/Details/a6...,https://inspectourcommunity.hnhss.ca/Portal/In...,https://inspectourcommunity.hnhss.ca/Portal/Fa...
15,0,1,Follow-up,15-Sep-2022,location.href = '/Portal/Inspection/Details/78...,https://inspectourcommunity.hnhss.ca/Portal/In...,https://inspectourcommunity.hnhss.ca/Portal/Fa...
16,0,4,Follow-up,30-Jun-2022,location.href = '/Portal/Inspection/Details/a8...,https://inspectourcommunity.hnhss.ca/Portal/In...,https://inspectourcommunity.hnhss.ca/Portal/Fa...
17,2,6,Follow-up,14-Jun-2022,location.href = '/Portal/Inspection/Details/e9...,https://inspectourcommunity.hnhss.ca/Portal/In...,https://inspectourcommunity.hnhss.ca/Portal/Fa...


In [11]:
final_df.to_csv( output_path + "2_level_table.csv", index=False)

In [13]:
#close webdriver
try:
    driver.close()
except:
    pass